***

# 4. Introduction to temperature logs

Given that geothermal development seeks to mine heat energy from the Earth, temperature logs are arguably the most important data we acquire. In this tutorial, we will give a short introduction to handling these data and how they are interpreted.

## 4.1 Uses of temperature logs in geothermal exploration and development

Feed zone identification (all raw well temperature logs)

Interpreting well/reservoir events, such as a change in the reservoir temperature with time such as heating after drilling or the initiation of a cool reservoir down flow (comparison of static temperature logs with time)

Constraining conceptual models and calibrating reservoir models (natural state temperature profiles)

## 4.2 Key considerations when evaluating temperature data

When the t log was acquired (e.g., the time that has elapsed since drilling or since the well was flowed) 

the well status was (shut, injection, flowing) while the log was acquired 

the conditions inside the well (liquid, two-phase, steam, cold gas)



***

# 5. Extract and evauluate a temprature aquired during the completion test

## 5.1 Use functions to import and mudge the PTS and pump data

The data munging process in 1-overview.ipynb has been turned into specilised functions located in the functions.py file. We import and use that set of functions in the same way as any other module.

In [ ]:
import pandas as pd
from utilities import* # functions in the utilities.py file



Because the cell below includes all steps required to import and mudge the data (everything from notebook 1), it will take a little while to run. 

In [ ]:
flowrate = read_flowrate(r'Data-FlowRate.xlsx')
pts = read_pts(r'Data-PTS.xlsx')

In [ ]:
print(pts.shape)
pts.head(2)

## 5.2 Look at the data

It is good practice to plot data after import so any issues are revealed. 

Note that the temprature logs look a little odd because they include the data aquired while the tool was held stationary for pressure monitoring. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(24,8),sharey=True)

spinner_scatter = ax1.scatter(pts.temp_degC, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)
datetime_scatter = ax2.scatter(pts.datetime, pts.depth_m, c = pts.timestamp, s = 5, linewidths = 0)

ax3 = ax2.twinx()
ax3.plot(flowrate.datetime, flowrate.flow_tph, 
    c='k', linestyle = '-', linewidth = 3, alpha = 0.3, 
    label='Surface pump flowrate')

ax1.set_ylabel('Depth [m]')
ax1.set_xlabel('Temp [degC]')

ax2.set_xlabel('Time [hh:mm]')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax3.set_ylabel('Flowrate [t/hr]')

ax1.set_ylim(1000,0)
ax1.set_xlim(0,200)

for ax in [ax1, ax2]:
    ax.grid()


## 5.3 Remove data aquired when the tool is stationary or slowing

We decided that, in this case, data acquired when the tool is moving down the well at > 0.02 m/s is fast enough to be included. 

Note that all data acquired while the tool is winched upward inside the well (negative numbers) are included and we have just set an arbitrary value for the filter so all these data are included. 

We use a Boolean expression to filter the pts dataframe. Our new working dataframe is called moving_pts

In [ ]:
moving_pts = pts[
    (pts.speed_mps > 0.9 ) & (pts.speed_mps < pts.speed_mps.max()) | 
    (pts.speed_mps > pts.speed_mps.min() ) & (pts.speed_mps < -0.9)
    ]

In [ ]:
moving_pts.describe()

## 5.4 Extract the first heating run from the data

We often want to select a single temperature log for analysis. In this example, we will extract the first heating run (i.e., the temperature log acquired after we stopped pumping into the well)

We will use an interactive tool called ipywidgets to select the start and end time of our desired data range. 

In [ ]:
from ipywidgets import interactive, Layout, FloatSlider

min_timestamp = pts.timestamp.iloc[0]
max_timestamp = pts.timestamp.iloc[-1]

def subselect_plot(start_value, stop_value):
    f,ax = plt.subplots(1,1, figsize = (20,6))
    ax.scatter(moving_pts.timestamp, moving_pts.depth_m,
        c = 'k', s = 1, linewidths = 0, label = 'Tool depth')
    ax1 = ax.twinx()
    ax1.plot(flowrate.timestamp, flowrate.flow_tph, 
        ':', c='k', label='Surface pump flowrate')
    ymin = pts.depth_m.min()
    ymax = pts.depth_m.max() + 100
    ax.vlines(start_value, ymin, ymax, color='tab:green')
    ax.vlines(stop_value, ymin, ymax, color='tab:red')
    ax.set_ylim(pts.depth_m.max() + 100, 0)
    ax.set_xlabel('Timestamp [sec]')
    ax.set_ylabel('Tool depth [m]')
    ax1.set_ylabel('Flowrate [t/hr]')

result = interactive(subselect_plot,
         
         start_value = FloatSlider
         (
             value = (max_timestamp - min_timestamp)/3 + min_timestamp,
             description = 'start',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=False,
             layout = Layout(width='80%'),
             ),
          
          stop_value = FloatSlider
          (
             value = (max_timestamp - min_timestamp)/2 + min_timestamp, 
             description = 'stop',
             min = min_timestamp, 
             max = max_timestamp, 
             step = 10, 
             continuous_update=False,
             layout = Layout(width='80%')
             )
)

display(result);


### Make your selection

Move the sliders above and then run the cell below to return the timestamp at the location of the slider. 

Each time you change this plot, the underlying array of data that we call with the result.children\[x\] statement is changed.

In [ ]:
print(
    'start =',result.children[0].value, ' which is', datetime.fromtimestamp(result.children[0].value), 
    '\n stop =', result.children[1].value, ' which is', datetime.fromtimestamp(result.children[1].value),
    )

### Record your analysis

As part of making this process repeatable and so that it is easy to come back later to check the analysis, you need to use the range you have selected to manually define objects in the next step.

Copy-paste the timestamps printed above into the cell below. They are now the objects that define the start and stop points for splicing the moving_pts dataframe. 

We could have defined the start and stop objects as the result.childern\[0\].value and result.childern\[1\].value objects. But if we did this, you would lose your work because these ipywidget results change every time the sliders are moved or the notebook is re-run. 

In [ ]:

start = 1607655784.448  # paste your start value here
stop = 1607656174.448   # paste your stop value here
name = 'First heating run'  # create an informative title for the data

pts_first_heating_run = moving_pts[
    (moving_pts.datetime > datetime.fromtimestamp(start)) 
    & (moving_pts.datetime < datetime.fromtimestamp(stop))
]

flowrate_first_heating_run = flowrate[
    (flowrate.datetime > datetime.fromtimestamp(start)) 
    & (flowrate.datetime < datetime.fromtimestamp(stop))
]

In [ ]:
pts_first_heating_run.shape

In [ ]:
# you can use this overview plot to check your data
#overview_fig(pts_first_heating_run, flowrate_first_heating_run, title = name).show()

## 5.5 Import and calculate contextualising data

We have provided you with some contextualising data that:
1. Illustrates how geothermal wells heat up over time 
2. Helps you interpret the temperature log we just extracted 

### 5.5.1 Stable temperature

A temperature log was acquired in this well 37 days after it was drilled and tested. In this resource, it reflects the stable temperature of this well. In low permeability settings, wells may take months to fully heat after drilling and testing.

In [ ]:
heating_37days = pd.read_csv('Data-heating37days.csv')

heating_37days['pressure_bara'] = heating_37days.pres_barg - 1

heating_37days.head(2)

### 5.5.2 Saturation temperature aka the boiling point for depth (BPD) curve

Water will boil at a given pressure - temperature condition. We use steam tables to calculate the temperature boiling will occur given the measured pressure. This curve helps us to understand the thermodynamic conditions inside the well. 

The python package we use to access the steam tables is not already in Anaconda. If you did not make an environment for this tutoral using the environment.yml file, you may need to install iapws. 

If you get an error saying **FILL THIS ERROR IN** when you run the cell below, then open the Anaconda prompt (windows) or a terminal (mac) and run the following after you have checked that you are in the correct conda environment.

> pip install iapws

In [ ]:
import iapws
# note that iapws uses SI units so some unit conversion is required

heating_37days['pressure_mpa'] = heating_37days.pressure_bara * 0.1  # convert pressure to MPa for ipaws

pressure = heating_37days['pressure_mpa'].tolist()
tsat = []
for p in pressure:
    saturation_temp = iapws.iapws97._TSat_P(p) - 273.15  # calculate saturation temp in Kelvin & convert to degC
    tsat.append(saturation_temp)
heating_37days['tsat_degC'] = tsat

heating_37days.head(2)

### 5.5.3 Well completion information

Understanding where we are in the well helps us to interpret the data. This is particularly important when we seek to understand what is a process occurring inside the well and what is related to the reservoir. 

#### Datums and depth measurements for well test data

The depth of downhole data comes in two forms: 
- Meters measured depth (mMD), which is the number of meters along the well path
- Vertical meters (mVD), which is the number of meters in the vertical dimension 

It is typical for completion test data to be acquired and analysed in measured depth. However, when we seek to understand features of the reservoir, we need to also consider data in vertical depth.  

There are a range of datums that are used in geothermal: 
- During drilling, most data is acquired relative to the rig floor or rotary table
- Well test data acquired after the rig has been demobilised is typically acquired relative to the casing head flange (CHF) or the cellar top (CT), where the latter typically has a survey marker
- Data may also be presented relative to sea level (RSL) or above sea level (ASL)

It is ambiguous to define data relative to ground level (GL) because of the degree of civil construction that occurs on well pads. 

In [ ]:
production_shoe = 462.5     # 13 3/8 production casing shoe in meters measured depth (mMD) from the casing head flange (CHF)
top_of_liner = 425          # top of perferated 10 3/4 liner in meters measured depth (mMD) from CHF
terminal_depth = 950  # NEED TO GET EXACT NUMBER FROM KATIE

## 5.6 Visualise the data for interpretation



In [ ]:
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates

fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,10),sharey=True)


ax1.set_title('All completion test temperature logs')

# all temp data colored by time 
ax1.scatter(moving_pts.temp_degC, moving_pts.depth_m,     
    c = moving_pts.timestamp, s = 5, linewidths = 0)

# our sub-selected data
ax1.plot(pts_first_heating_run.temp_degC, pts_first_heating_run.depth_m,
    color = 'k', linestyle = ':', label = 'Our selected data')

# blank well casing
ax1.plot([1, 1],[0, production_shoe],
    color = 'k', linewidth = 3, linestyle = '-')

# perforated well casing
ax1.plot([5, 5],[top_of_liner, terminal_depth],     
    color = 'k', linewidth = 1.5, linestyle = '--')


ax2.set_title('Heating (shut) temperature logs')

# our sub-selected pressure + temp data
ax2.plot(pts_first_heating_run.temp_degC, pts_first_heating_run.depth_m, 
    color ='#EBE85B', linewidth = 3, label = 'Day 0 - temp')

ax2.plot(pts_first_heating_run.pressure_bara, pts_first_heating_run.depth_m, 
    linestyle = '--', linewidth = 3, color = '#EBE85B', label = 'Day 0 - pres')

# stable pressure + temp data
ax2.plot(heating_37days.temp_degC, heating_37days.depth_m, 
    color = 'k', label = 'Day 37 - temp')

ax2.plot(heating_37days.pressure_bara, heating_37days.depth_m, 
    linestyle = '--', color = 'k', label = 'Day 37 - pres')

# saturation temp for the stable pressure
ax2.plot(heating_37days.tsat_degC, heating_37days.depth_m, 
    linestyle = ':', color = 'k', label = 'Day 37 - BPD')


ax1.set_ylim(1000,0)
ax1.set_ylabel('Depth [m]')
ax1.set_xlim(0,200)
ax1.legend()

ax2.legend()
ax2.set_xlim(0,300)

for ax in [ax1,ax2]:
    ax.set_xlabel('Temperature [degC] or Pressure [bara]')
    ax.grid()


## 5.7 Export the first heating run as a csv 

We use temperature to help us identify feedzones. We export our selected data so we can use it in the spinner analysis.

In [ ]:
pts_first_heating_run.to_csv('data-heating0days.csv', index=False)

This is not a course on how to read temprature logs. However, we can make a couple of quick observations of these data. 

WRITE SOME INTERPRETIVE COMMENTS HERE

- notice how the rate of heating is not even along the wellpath after we shut the pumps in
- notice how there is a point on the well where the temprature steps when we are injecting (fluid is leaving the well)

***

<p><center>© 2021 <a href="https://www.cubicearth.nz/">Irene Wallis</a> and <a href="https://www.linkedin.com/in/katie-mclean-25994315/">Katie McLean</a> <a href="https://creativecommons.org/licenses/by/4.0/"</a></center></p>

<p><center>Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a></center></p>

<p><center>Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.</center></p>

***